In [1]:
import pandas as pd
import datetime as dt
import re
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
bets = pd.read_csv('past_bets/base/bets_20220713.csv', index_col = 0)

In [3]:
bets = bets[bets.Home_Team != 'Los Angeles Angels']
bets.reset_index(drop = True, inplace = True)
bets

,Home_Team,Away_Team,Home_Prob,Away_Prob,Home_ML,Away_ML,Home_ML_Prob,Away_ML_Prob,Home_Diff,Away_Diff,Home_KC,Away_KC,Home_Bet,Away_Bet
0,Washington Nationals,Seattle Mariners,0.470337,0.529663,-116.0,-102.0,0.537037,0.504950,-0.066700,0.024712,0.000000,0.004992,0.000000,586.657004
1,Atlanta Braves,New York Mets,0.657581,0.342419,-156.0,132.0,0.609375,0.431034,0.048206,-0.088615,0.012341,0.000000,1450.296033,0.000000
2,Minnesota Twins,Milwaukee Brewers,0.644271,0.355729,-126.0,108.0,0.557522,0.480769,0.086749,-0.125040,0.019605,0.000000,2304.050070,0.000000
3,Kansas City Royals,Detroit Tigers,0.507618,0.492382,-124.0,106.0,0.553571,0.485437,-0.045953,0.006945,0.000000,0.001350,0.000000,158.620962
4,San Francisco Giants,Arizona Diamondbacks,0.571799,0.428201,-126.0,108.0,0.557522,0.480769,0.014277,-0.052568,0.003227,0.000000,379.196294,0.000000
5,Miami Marlins,Pittsburgh Pirates,0.605988,0.394012,-178.0,150.0,0.640288,0.400000,-0.034300,-0.005988,0.000000,0.000000,0.000000,0.000000
6,New York Yankees,Cincinnati Reds,0.910065,0.089935,-335.0,270.0,0.770115,0.270270,0.139950,-0.180336,0.043485,0.000000,5110.401037,0.000000
7,Toronto Blue Jays,Philadelphia Phillies,0.677883,0.322117,-118.0,100.0,0.541284,0.500000,0.136599,-0.177883,0.029779,0.000000,3499.637826,0.000000
8,Tampa Bay Rays,Boston Red Sox,0.666862,0.333138,-196.0,164.0,0.662162,0.378788,0.004700,-0.045650,0.001391,0.000000,163.487049,0.000000
9,Cleveland Guardians,Chicago White Sox,0.534546,0.465454,-104.0,-112.0,0.509804,0.528302,0.024742,-0.062848,0.005047,0.000000,593.188210,0.000000


In [4]:
bets.to_csv('past_bets/base/bets_20220713.csv')